# Lecture 11

## Feature Engineering <a class="anchor" id="TOC"></a>

### [Part I](#part1): World-Management Survey Data 
                                                               
  - Creating new variable(s) from multiple           
       already existing (mean of multiple variable)  
  - Grouping a categorical variable:                 
       countries to continents                       
  - Ordered variables:                               
     - creating an ordered factor                    
         from character or integer                   
     - creating an ordered                           
         from numeric                                
  - Factors or dummy variables:                      
       creating multiple dummies                     
  - Extra: intro to principal component analysis     
                                                     
                                                     
### [PART II](#part2): Bisnode Data                                   
                                                     

- imputing:                                     
    - A: replacing with mean or median            
    - B: outside knowledge to replace values      
    - C: introduce new value:                     
        - only for categorical values           
- log transformation adjustment:                
        log(0) is -Inf -> adjust numerically        
- create dummy variable(s) with                 
        multiple statements: using lead() function  
- randomizing large data for visualization      
- growth rate with log difference:
    - using lag() function                      
- winsorizing                                   

___

Import packages

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import warnings

%matplotlib inline
warnings.filterwarnings("ignore")

## Part I<a class = 'anchor' id = 'part1'></a>

 World-Management Survey Data
  - Creating new variable(s) from multiple already existing (mean of multiple variable)
  - Grouping a categorical variable: countries to continents
  - Ordered variables:
     - creating an ordered factor variable from character or integer variables
     - creating an ordered variable from numeric
  - Factors or dummy variables: creating multiple dummies

Import World-Management Survey Data

In [ ]:
wms  = pd.read_csv("https://osf.io/uzpce/download")

wms.head()

### Creating a continuous variable out of ordered variables

Trick: lean, perf and talent measures, but multiple variables.\
    1. `filter(regex=)` will select these variables.
    2. calculate average with `mean(axis=1)` for each observation

In [ ]:
wms["avg_score"] = wms.filter(regex="lean|perf|talent").mean(axis=1)

In [ ]:
wms["avg_score"].describe()

#### Task:
   create the sum of `aa_` variables \
   check that the resulting variable has value of 1 for each observation as `aa_` variables are dummies for industry code

In [ ]:
wms["sum_aa"] = wms.filter(regex="aa_").sum(axis=1)
wms["sum_aa"].describe()

### Grouping categorical

Creating groups by continents -> reducing dimensionality of a categorical variable


In [ ]:
wms["country"].value_counts()

In [ ]:
wms["country"].value_counts(normalize = True)

`pycountry_convert` module converts country names to country codes and continents

In [ ]:
import pycountry_convert as pc

Note: Norther Ireland is not in this database, so convert it by hand. Also, Ireland has to be trimmed.

In [ ]:
wms["continent"] = (
    wms["country"]
    .apply(lambda x: np.where(x == "Northern Ireland", "Ireland", x))
    .apply(lambda x: np.where(x == "Republic of Ireland", "Ireland", x))
    .apply(pc.country_name_to_country_alpha2) # converts country name to country code
    .apply(pc.country_alpha2_to_continent_code) # country code to continent code
    .apply(pc.convert_continent_code_to_continent_name)# continent code to name
)

In [ ]:
wms["continent"].value_counts(dropna=False)

 It is also possible to create these groups by hand, with `np.where` command.

In [ ]:
wms["ownership"].value_counts(dropna=False)

In [ ]:
wms["owner"] = np.where(
    wms["ownership"].isnull(),
    np.nan,
    np.where(
        wms["ownership"] == "Government",
        "govt",
        np.where(
            wms["ownership"].str.contains("family", regex=False),
            "family",
            np.where(wms["ownership"] == "Other", "other", "private"),
        ),
    ),
)

In [ ]:
wms["owner"].value_counts(dropna=False)

### Good-to-know: labeled ordered categorical variable: 
labels are ordered, however difference is only in few application

In [ ]:
wms["lean1_ord"] = pd.cut(
    wms["lean1"], 5, labels=["extremly poor", "bad", "mediocre", "good", "excellent"]
)

Can easily plot

In [ ]:
(
    ggplot(wms, aes(x="lean1_ord", y="avg_score"))
    + stat_summary(geom="point", fun_data="mean_se", size=8, fill="red")
    + labs(x="Lean 1 score", y="Mean average management score")
    + theme_bw()
)

#### Task:
Create the same graph, but using the `talent2` variable instead

In [ ]:
wms["talent2_ord"] = pd.cut(
    wms["talent2"], 5, labels=["extremly poor", "bad", "mediocre", "good", "excellent"]
)

In [ ]:
(
    ggplot(wms, aes(x="talent2_ord", y="avg_score"))
    + stat_summary(geom="point", fun_data="mean_se", size=8, fill="red")
    + labs(x="Talent 1 score", y="Mean average management score")
    + theme_bw()
)

##### Numeric to ordered

It is hard to get any conclusion if we plot the pattern between 
   average management score and number of employees

In [ ]:
(
    ggplot(wms, aes(x="emp_firm", y="avg_score"))
    + geom_point(color="red", size=2, alpha=0.6)
    + labs(x="Number of employees", y="Mean average management score")
    + theme_bw()
)

One simple way to solve this issue:\
Simplifying firm size: creating categories from numeric

In [ ]:
wms["emp_cat"] = pd.cut(
    wms["emp_firm"], bins=[0, 200, 1000, np.inf], labels=["small", "medium", "large"]
)

In [ ]:
(
    ggplot(
        wms.loc[
            lambda x: x["emp_cat"].notnull(),
        ],
        aes(x="emp_cat", y="avg_score"),
    )
    + stat_summary(geom="point", fun_data="mean_se", size=8, fill="red", na_rm=True)
    + labs(x="Firm size", y="Mean average management score")
    + theme_bw()
)

### Factors Or Dummies

Creating multiple factor dummy from a categorical

In [ ]:
dummies = pd.get_dummies(wms["emp_cat"], dummy_na = True)
dummies

You can easily concatenate this to the original dataframe

In [ ]:
wms = pd.concat([wms,dummies],axis=1)
wms.head()

### Extra:

principle component analysis or PCA

One can argue, that the mean of the score is not the best measure, as it takes each value with the same weight \
An alternative solution is creating principal components, which transform the original variables.

import PCA function from sklearn

In [ ]:
from sklearn.decomposition import PCA

Let us create principle components with all the questionnaires. \
have to make sure there is no NA value

In [ ]:
original_variables = wms.filter(regex="lean|perf|talent").filter(regex="^(?!.*ord).*$").dropna()
original_variables.shape

fit PCA model

In [ ]:
pca = PCA()

pca.fit(original_variables)

We have the same number of variables, but they are transformed.

As PCA is an information reductionist approach, we can see, 
     which transformed variable explains what percent of the overall information (variation)

In [ ]:
pca.explained_variance_ratio_

Let us decide to use only the first variable, which explains 45.6%

In [ ]:
pca_components = pd.DataFrame(
    pca.fit_transform(original_variables),
    columns=["PC%s" % str(i + 1) for i in range(len(original_variables.columns))],
)
pca_components.shape

aux: add firmid and wave with same filter to match PCs to wms data

In [ ]:
aux = (
    wms.filter(regex="lean|perf|talent|wave|firmid")
    .filter(regex="^(?!.*ord).*$")
    .dropna()
    .filter(["wave", "firmid"])
    .reset_index(drop=True)
)
aux.shape

add firmid wave and only PC0 from pca-s

In [ ]:
pca_dataframe = pd.concat([aux, pca_components["PC1"]], axis=1)

pca_dataframe.shape

add to wms data

In [ ]:
wms = wms.merge(pca_dataframe, on = ["firmid","wave"],how="left")

Compare descriptives with average score


In [ ]:
wms.filter(["avg_score", "PC1"]).describe()

Create a bin-scatter with PC1

In [ ]:
(
    ggplot(
        wms.loc[
            lambda x: x["emp_cat"].notnull(),
        ],
        aes(x="emp_cat", y="PC1"),
    )
    + stat_summary(geom="point", fun_data="mean_se", size=8, fill="red", na_rm=True)
    + labs(x="Firm size", y="Principal component")
    + theme_bw()
)

Notes: 
  1) PCA is especially useful when you have too many explanatory variables and want to reduce num vars, 
      with minimal information loss. However, should use it with care, especially with time series! \
  2) There are many variations of PCA, if one starts to `rotate` the factors 
      to make some meaningful variables out of it (especially in psychology) \
  3) There are many packages, which carry out PCA, this is pretty much the simplest intro here... \

## Part II<a class = 'anchor' id = 'part2'></a>

Bisnode data to show real-life situations for:
  - imputing: 
      - A: replacing with mean or median
      - B: using outside knowledge to replace values
      - C: introduce new value -> only for categorical values
  - log transformation adjustment: log(0) is -Inf -> adjust numerically
  - create dummy variable(s) with multiple statements: using lead() function
  - randomizing large data for visualization
  - growth rate with log difference: using lag() function
  - winsorizing

Using bisnode data for firm exit

In [ ]:
bisnode = pd.read_csv("https://osf.io/3qyut/download")

bisnode.head()

Sample selection\
drop variables with many NAs

In [ ]:
bisnode = bisnode.drop(
    ["COGS", "finished_prod", "net_dom_sales", "net_exp_sales", "wages"], axis=1
).loc[bisnode["year"] != 2016]

add all missing year and comp_id combinations -

   
(originally missing combinations will have NAs in all other columns)

In [ ]:
bisnode = (
    bisnode.set_index(["year", "comp_id"])
    .unstack(fill_value=np.nan)
    .stack(dropna=False)
    .reset_index()
)

### Imputing

A) Replacing with mean or median:
    
   number of employed in firm is a noisy measure with many missing value.\
   replace missing values with the mean or median\
   also add a flag variable for the imputed values (need to include in the model!)

In [ ]:
# mean
bisnode["labor_avg_mod"] = np.where(
    bisnode["labor_avg"].isnull(),
    np.nanmean(bisnode["labor_avg"]),
    bisnode["labor_avg"],
)
# median
bisnode["labor_med_mod"] = np.where(
    bisnode["labor_avg"].isnull(),
    np.nanmedian(bisnode["labor_avg"]),
    bisnode["labor_avg"],
)
# flag
bisnode["flag_miss_labor_avg"] = bisnode["labor_avg"].isnull()

#### Task
add `Nmiss` as a custom function to datasummary and check the \
mean, median, sd, N and Nmiss for labor_avg, labor_avg_mod, labor_med_mod

In [ ]:
def Nmiss(x):
    return x.isnull().sum()

Check how stats altered, discuss!

In [ ]:
bisnode.filter(["labor_avg", "labor_avg_mod", "labor_med_mod"]).agg(
    ["mean", "median", "std", "count", Nmiss]
).T

### Imputing:

B) Using outside knowledge to replace values:

Negative sales should not happen, thus we can overwrite it to a small value: 1

In [ ]:
bisnode["sales"].describe()

In [ ]:
bisnode["sales"] = np.where(bisnode["sales"] < 0, 1, bisnode["sales"])

In [ ]:
bisnode["sales"].describe()

### Imputing:

C) Categorical variables

Simplify some industry category codes and set missing values to 99

In [ ]:
bisnode["ind2_cat"] = np.where(bisnode["ind2"] > 56, 60, bisnode["ind2"])
bisnode["ind2_cat"] = np.where(bisnode["ind2"] < 26, 20, bisnode["ind2_cat"])
bisnode["ind2_cat"] = np.where(
    (bisnode["ind2"] < 55) & (bisnode["ind2"] > 35), 40, bisnode["ind2_cat"]
)
bisnode["ind2_cat"] = np.where(bisnode["ind2"] == 31, 30, bisnode["ind2_cat"])
bisnode["ind2_cat"] = np.where(bisnode["ind2"].isnull(), 99, bisnode["ind2_cat"])

In [ ]:
bisnode["ind2_cat"].value_counts().sort_index()

___

Adjusting negative sale and for log transformation:

In [ ]:
bisnode["ln_sales"] = np.where(bisnode["sales"] > 0, np.log(bisnode["sales"]), 0)
bisnode["sales_mil"] = bisnode["sales"] / 10**6
bisnode["sales_mil_log"] = np.where(bisnode["sales"] > 0, np.log(bisnode["sales_mil"]), 0)

***Creating 'status_alive' variable to decide if firm exists or not***

Generate status_alive; if sales larger than zero and not-NA, then firm is alive

In [ ]:
bisnode["status_alive"] = np.where(
    (bisnode["sales"] > 0) & (bisnode["sales"].notnull()), 1, 0
)

Defaults in two years if there are sales in this year but no sales two years later

In [ ]:
bisnode = bisnode.sort_values(by=["comp_id","year"])

bisnode["default"] = bisnode.groupby("comp_id")["status_alive"].transform(
    lambda x: (x == 1) & (x.shift(2) == 0)
).astype(int)

Select years before 2013

In [ ]:
bisnode = bisnode.loc[bisnode["year"]<= 2013]

To speed up let take a randomly selected 5k companies

In [ ]:
comp_id_f = bisnode.drop_duplicates(subset=["comp_id"]).sample(5000, random_state = 20123123)["comp_id"]

In [ ]:
bisnode_sample = bisnode.loc[lambda x: x["comp_id"].isin(comp_id_f)]

### Numeric vs Factor Representation

Numeric representation (good)

In [ ]:


(
    ggplot(bisnode_sample, aes(x="sales_mil_log", y="default"))
    + geom_point(size=2, alpha=0.3, color="blue")
    + geom_smooth(method="lm", formula="y ~ x**2", color="black", se=False, size=1)
    + geom_smooth(method="loess", se=False, colour="red", size=1.5)
    + labs(x="sales_mil_log", y="default")
    + theme_bw()
)

#### Task
convert default to a factor variable and plot!\
what is the problem? It is a bad idea to convert to a factor?

In [ ]:
bisnode_sample["default_factor"] = bisnode_sample["default"].astype("category")

(
    ggplot(bisnode_sample, aes(x="sales_mil_log", y="default_factor"))
    + geom_point(size=2, alpha=0.3, color="blue")
    + geom_smooth(method="lm", formula="y ~ x**2", color="black", se=False, size=1)
    + geom_smooth(method="loess", se=False, colour="red", size=1.5)
    + labs(x="sales_mil_log", y="default")
    + theme_bw()
)

Growth (%) in sales \
Take the lags but make sure only for the same company!

In [ ]:
bisnode["d1_sales_mil_log"] = bisnode.groupby("comp_id")["sales_mil_log"].transform(
    lambda x: x - x.shift(1)
)

Repeat random sample to include the new variables

In [ ]:
bisnode_sample = bisnode.loc[lambda x: x["comp_id"].isin(comp_id_f)]

 First measure for change in sales: take the sale change in logs

In [ ]:
nw = (
    ggplot(bisnode_sample, aes(x="d1_sales_mil_log", y="default"))
    + geom_point(size=1, fill="blue", color="blue")
    + geom_smooth(method="loess", se=False, colour="red", size=1.5)
    + labs(x="Growth rate (Diff of ln sales)", y="default")
    + theme_bw()
    + scale_x_continuous(limits=(-6, 10), breaks=np.arange(-5, 10, 5))
)
nw

### Winsorized Data:
  - set (extreme) values to a certain (lower) value

Note: 
    
 winsorizing is the action to set manually a value \
      'censoring' is called if the values are already 'winsorized' \
      thus it is unknown what was the original value, but can only see the set value \
        e.g. mother's wage who are at home is 0, however if she would work this value would be different \
      'truncation' is when we dropping certain values below or above a threshold from the data 

Create new variable and add flag variables for modelling

In [ ]:
bisnode["flag_low_d1_sales_mil_log"] = np.where(
    bisnode["d1_sales_mil_log"] < -1.5, 1, 0
)
bisnode["flag_high_d1_sales_mil_log"] = np.where(
    bisnode["d1_sales_mil_log"] > 1.5, 1, 0
)
bisnode["d1_sales_mil_log_mod"] = np.where(
    bisnode["d1_sales_mil_log"] < -1.5,
    -1.5,
    np.where(bisnode["d1_sales_mil_log"] > 1.5, 1.5, bisnode["d1_sales_mil_log"]),
)

Repeat random sample to include the new variables

In [ ]:
bisnode_sample = bisnode.loc[lambda x: x["comp_id"].isin(comp_id_f)]

In [ ]:
w = (
    ggplot(bisnode_sample, aes(x="d1_sales_mil_log_mod", y="default"))
    + geom_point(size=1, fill="blue", color="blue")
    + geom_smooth(method="loess", se=False, colour="red", size=1.5)
    + labs(x="Growth rate (Diff of ln sales)", y="default")
    + theme_bw()
    + scale_x_continuous(limits=(-1.5, 1.5), breaks=np.arange(-1.5, 1.51, 0.5))
)
w

#### Task:
Show the effect of winsorizing: transformation of the original data\
put d1_sales_mil_log on x-axis and d1_sales_mil_log_mod to the y-axis

In [ ]:
(
    ggplot(bisnode_sample, aes(x="d1_sales_mil_log", y="d1_sales_mil_log_mod"))
    + geom_point(size=1, fill="blue", color="blue")
    + labs(
        x="Growth rate (Diff of ln sales) (original)",
        y="Growth rate (Diff of ln sales) (winsorized)",
    )
    + theme_bw()
    + scale_x_continuous(limits=(-5, 5), breaks=np.arange(-5, 5, 1))
    + scale_y_continuous(limits=(-3, 3), breaks=np.arange(-3, 3, 1))
)